# Lesson 14 - Dynamic Polymorphism

This notebook supports the materials in [lesson 14 of C++ for Finance](https://loz-hurst.github.io/cpp-finance-materials-new/lessons/lesson-14.html).

## Forward declarations

It is a good idea to include a reference to where the full definition can be found in the comment, so save you searching for it later.

In [ ]:
// This class is in MyClasses.hpp
class ForwardDeclaredClass;

In [ ]:
void SomeFunction(ForwardDeclaredClass & cls) {
    // Do something that does not "use" ForwardDeclaredClass
}

You cannot do anything that requires knowing the insides of the class if you only have a forward declaration.  This will cause a compile-time error:

In [ ]:
void SomeOtherFunction(ForwardDeclaredClass & cls) {
    cls.SomeFunction();
}

Be careful of subtle cases where you are using (as opposed to referencing) a class, and therefore cannot use a forward declaration.  This is also a compile-time error because passing by value involves creating (using a constructor) a new instance:

In [ ]:
void SomeOtherFunction(ForwardDeclaredClass cls) {
    // Do something that does not "use" ForwardDeclaredClass
}

Even if your object uses the default constructor, C++ needs the definition to know for sure that another constructor has not been provided.

## Polymorphism

### Static and dynamic type, and virtual functions refresher

In [ ]:
#include <iostream>

In [ ]:
// A class
class MyClass {
public:
    // A pure virtual function
    virtual void VirtualFunction() const = 0;
    // A function that is not virtual
    void NotVirtualFunction() const {std::cout << "MyClass::NotVirtualFunction" << std::endl;}
};

// A class derived from MyClass
class MyDerivedClass : public MyClass {
public:
    // Implremented virtual function
    virtual void VirtualFunction() const {std::cout << "Derived::VirtualFunction" << std::endl;}
    // Override a function that is not virtual
    void NotVirtualFunction() const {std::cout << "Derived::NotVirtualFunction" << std::endl;}
};

// Another class derived from MyClass
class MyOtherDerivedClass : public MyClass {
public:
    // Implremented virtual function
    virtual void VirtualFunction() const {std::cout << "OtherDerived::VirtualFunction" << std::endl;}
    // Override a function that is not virtual
    void NotVirtualFunction() const {std::cout << "OtherDerived::NotVirtualFunction" << std::endl;}
};

In [ ]:
// Calls 'VirtualFunction' on the passed in reference
void CallVirtualFunction(MyClass & cls) {
    // Static type of cls is always "MyClass"
    cls.VirtualFunction();
}

In [ ]:
// Calls 'NotVirtualFunction' on the passed in reference
void CallNotVirtualFunction(MyClass & cls) {
    cls.NotVirtualFunction();
}

In [ ]:
MyDerivedClass derived_class;
MyOtherDerivedClass other_derived_class;

// Dynamic type of cls in CallVirtualFunction will be MyDerivedClass
CallVirtualFunction(derived_class);
CallNotVirtualFunction(derived_class);

// Dynamic type of cls in CallVirtualFunction will be MyOtherDerivedClass
CallVirtualFunction(other_derived_class);
CallNotVirtualFunction(other_derived_class);

In [ ]:
MyClass my_class; // Error - MyClass is abstract and cannot be created
// If it was not abstract, then the dynamic type of cls in CallVirtualFunction would be MyClass (same as its static type)
CallVirtualFunction(my_class);
CallNotVirtualFunction(my_class);